In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [3]:

# Read the text file
with open('sample_data/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [5]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [7]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [8]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [9]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))


In [10]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 95s 31ms/step - loss: 6.2317 - accuracy: 0.0767
Epoch 2/100
3010/3010 [==============================] - 94s 31ms/step - loss: 5.5106 - accuracy: 0.1242
Epoch 3/100
3010/3010 [==============================] - 93s 31ms/step - loss: 5.1312 - accuracy: 0.1458
Epoch 4/100
3010/3010 [==============================] - 96s 32ms/step - loss: 4.8014 - accuracy: 0.1646
Epoch 5/100
3010/3010 [==============================] - 95s 32ms/step - loss: 4.4961 - accuracy: 0.1824
Epoch 6/100
3010/3010 [==============================] - 94s 31ms/step - loss: 4.2061 - accuracy: 0.2035
Epoch 7/100
3010/3010 [==============================] - 94s 31ms/step - loss: 3.9288 - accuracy: 0.2277
Epoch 8/100
3010/3010 [==============================] - 94s 31ms/step - loss: 3.6669 - accuracy: 0.2584
Epoch 9/100
3010/3010 [==============================] - 94s 31ms/step - loss: 3.4202 - accuracy: 0.2915
Epoch 10/100
3010/3010 [==============================]

In [17]:
seed_text = "I will leave if they marry"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 19ms/step
I will leave if they marry before he go
